<a href="https://colab.research.google.com/github/Henriqueqb/Henrique/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents="Quando é a proxima imersão IA com Google Gemini da Alura?"
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar a página da Alura:** Fique de olho na página principal da Alura e na seção de cursos de Inteligência Artificial. Eles costumam anunciar novas edições por lá.
*   **Seguir as redes sociais da Alura:** Siga a Alura no LinkedIn, Instagram, e outras plataformas. Eles frequentemente divulgam novidades e lançamentos por lá.
*   **Assinar a newsletter da Alura:** Ao assinar a newsletter, você receberá informações sobre novos cursos e eventos diretamente no seu e-mail.

Você pode encontrar mais informações e se inscrever na newsletter no site da Alura: [https://www.alura.com.br/](https://www.alura.com.br/)

In [7]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools": [{"google_search":{}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio.


In [8]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com



In [9]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [13]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa e sua tarefa é usar a ferramenta de busca do google (google search) para recuperar as ultimas
        noticias de lançamentos muito relevantes sobre o topico abaixo.
        Foque no maximo em 4 lançamentos relevantes, com base na quantidade e entusiasmo das noticias sobre ele.
        Se um tema tiver poucas noticias ou reações entusiasmadas, é possivel que não seja um tema tão relevante, e pode ser substituido por outro que tenha mais.
        Esses lançamentos relevantes devem  ser atuais, de no maximo 30 dias antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools={google_search}
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)

    return lancamentos

In [14]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta do google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderiamos abordar em um post, sobre
        cada um deles. Você tambem pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano
        com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [24]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para um site de notícias sobre novidades no munda da tecnologia.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho_do_post = call_agent(redator, entrada_do_agente_redator)
    return rascunho_do_post

In [22]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor erRevisor de conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    post_final = call_agent(revisor, entrada_do_agente_revisor)
    return post_final

In [23]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esque ceu de digitar o topico!")
else:
  print(f"Otimo! Vamos criar o post sobre as novidades do {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do agente 1 (buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("-------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do agente 2 (planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("-------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do agente 3 (redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("-------------------")

  texto_revisado = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do agente 4 (revisor) ---\n")
  display(to_markdown(texto_revisado))

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: pokemon


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-76' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

Otimo! Vamos criar o post sobre as novidades do pokemon

--- Resultado do agente 1 (buscador) ---



> Para encontrar os lançamentos mais relevantes de Pokémon nos últimos 30 dias, vou realizar algumas pesquisas no Google.
> 
> 
> Com base nas notícias e lançamentos recentes de Pokémon nos últimos 30 dias, aqui estão alguns dos eventos e novidades mais relevantes:
> 
> 1.  **Eventos em Pokémon GO:**
> 
> *   **Duelo da Realeza:** Destaque para a estreia de Kingambit (21 de abril de 2025).
> *   **Evento Crescendo e Aprendendo:** Com aumento de PE ao evoluir Pokémon e bônus ao chocar ovos (início em 2 de maio de 2025).
> *   **Dia Comunitário:** Destaque para Pawmi em abril e eventos futuros em maio.
> *   **Retorno de Pokémon Dinamax:** Entei e Suicune Dinamax em reides (abril/maio de 2025).
> *   **Equipe GO Rocket:** Eventos com Regigigas Sombroso e Dialga Sombroso.
> *   **Caminho da Recompensa de Maio:** Recompensas ao comprar na loja do jogo e loja online.
> 
> 2.  **Pokémon TCG Pocket:**
> 
> *   Expansão Luz Triunfante com Oricorio e Magnezone (Maio de 2025).
> *   Promoções com cartas lendárias da Equipe Rocket (Articuno e Zapdos).
> 
> 3.  **Pokémon Legends: Z-A:**
> 
> *   Confirmação de lançamento para 2025 (Nintendo Switch e mobile).
> 
> 4.  **Pokémon Champions:**
> 
> *   Novo jogo para Switch e mobile sem data de lançamento definida.
> 
> Esses lançamentos e eventos parecem ser os mais comentados e relevantes no universo Pokémon neste momento.


-------------------

--- Resultado do agente 2 (planejador) ---



> Okay, com base nos lançamentos recentes de Pokémon, vou criar um plano de conteúdo abordando os pontos mais relevantes para um post de redes sociais. Para isso, usarei a ferramenta do Google para aprofundar as informações sobre cada um dos tópicos.
> 
> 
> Com base nas minhas pesquisas, o tema mais relevante para um post de redes sociais sobre Pokémon nos últimos 30 dias é **Pokémon GO: Duelo da Realeza e Invasão da Equipe Rocket**.
> 
> **Justificativa:**
> 
> *   **Pokémon GO é extremamente popular:** Pokémon GO continua sendo um dos jogos mobile mais jogados, garantindo alto engajamento.
> *   **Eventos simultâneos:** A combinação dos eventos "Duelo da Realeza" e "Invasão da Equipe Rocket" cria um momento dinâmico e cheio de oportunidades para os jogadores.
> *   **Novidades atrativas:** A estreia de Kingambit, o retorno de Dialga Sombroso e a possibilidade de remover o ataque Frustração de Pokémon Sombrosos são incentivos fortes para os jogadores participarem.
> 
> **Tópicos Relevantes:**
> 
> *   Estreia de Kingambit no evento Duelo da Realeza
> *   Destaque para Nidoqueen e Nidoking com coroas (versões brilhantes)
> *   Invasão da Equipe Rocket e Dialga Sombroso
> *   Possibilidade de remover o ataque Frustração
> *   Regigigas Sombroso nas reides sombrias
> *   Dia comunitário de Pawmi
> 
> **Plano de Conteúdo:**
> 
> 1.  **Título:** Prepare-se para a Realeza Sombria no Pokémon GO! 👑😈
> 2.  **Introdução:**
> 
>     *   Um breve resumo dos eventos "Duelo da Realeza" e "Invasão da Equipe Rocket".
>     *   Destaque a emoção de ter dois eventos acontecendo ao mesmo tempo.
> 3.  **Duelo da Realeza:**
> 
>     *   Apresentação de Kingambit: como evoluir, habilidades e dicas para usá-lo em batalhas.
>     *   Nidoqueen e Nidoking com coroas: detalhes sobre como encontrá-los e a chance de obter as versões brilhantes.
> 4.  **Invasão da Equipe Rocket:**
> 
>     *   Retorno de Dialga Sombroso: informações sobre como enfrentar Giovanni e capturar Dialga Sombroso.
>     *   Remoção do ataque Frustração: um guia passo a passo de como remover o ataque Frustração de Pokémon Sombrosos e por que isso é importante.
>     *   Novos Pokémon Sombrosos e reides: destaque os novos Pokémon Sombrosos disponíveis e dicas para vencer as reides.
> 5.  **Outros Eventos:**
> 
>     *   Dia comunitário de Pawmi: data, horário, bônus e dicas para aproveitar ao máximo o evento.
>     *   Regigigas Sombroso nas reides sombrias
> 6.  **Chamada para Ação:**
> 
>     *   Incentive os seguidores a participar dos eventos e compartilhar suas experiências.
>     *   Faça perguntas para estimular o engajamento (ex: "Qual Pokémon Sombroso você está mais ansioso para ter?", "Você já conseguiu um Kingambit?").
> 7.  **Recursos Visuais:**
> 
>     *   Imagens e vídeos dos eventos, dos Pokémon em destaque e das batalhas.
>     *   Memes e GIFs relacionados aos eventos e aos Pokémon.
> 
> Este plano de conteúdo equilibrado entre informações úteis e elementos de engajamento garantirá que o post seja informativo, divertido e relevante para a comunidade Pokémon.
> 


-------------------

--- Resultado do agente 3 (redator) ---



> 👑😈 Prepare-se para a Realeza Sombria no Pokémon GO! 👑😈
> 
> Dois eventos ÉPICOS rolando ao mesmo tempo? 😱 Prepare suas pokebolas porque o "Duelo da Realeza" e a "Invasão da Equipe Rocket" vão te deixar sem fôlego! 🔥
> 
> 👑 Duelo da Realeza:
> 
> *   Kingambit chegou! 👑 Aprenda a evoluir seu Bisharp e domine as batalhas com esse Pokémon incrível!
> *   Nidoqueen e Nidoking de Coroa: ✨ Já pensou em ter essas majestades brilhantes? 🤩 A hora é agora!
> 
> 😈 Invasão da Equipe Rocket:
> 
> *   Dialga Sombroso está de volta! 🚀 Encare o Giovanni e adicione esse lendário ao seu time!
> *   ATAQUE FRUSTRAÇÃO? NUNCA MAIS! 💪 Descubra como remover essa habilidade e turbinar seus Pokémon Sombrosos!
> *   E não para por aí! Regigigas Sombroso nas reides sombrias e novos Pokémon Sombrosos te esperam!
> 
> 🐾 Bônus:
> 
> *   Dia Comunitário de Pawmi chegando! Fique ligado para não perder!
> 
> E aí, qual Pokémon Sombroso você está mais ansioso para capturar? Conta pra gente nos comentários! 👇
> 
> #PokemonGO #EquipeRocket #Kingambit #Alura
> 


-------------------

--- Resultado do agente 4 (revisor) ---



> Olá! 👋 Como seu editor e revisor particular, dei uma olhada no seu rascunho. Achei que ele captura bem a empolgação dos eventos e fala direto com a galera! 😉
> 
> Porém, para sermos super claros e precisos (como um bom revisor deve ser! ✨), tenho algumas sugestões:
> 
> 1.  **Título:** "Realeza Sombria" mistura um pouco os temas dos dois eventos principais que, apesar de simultâneos, têm focos diferentes (Realeza: Kingambit, Nidos; Sombrosos: Team Rocket, Shadow Pokémon). Para evitar confusão e destacar que são *dois* eventos fortes, que tal um título que deixe isso mais claro?
>     *   Sugestão: "Dois Eventos ÉPICOS em Pokémon GO! Realeza e Invasão Sombria rolando! 👑😈"
>     *   Sugestão 2: "Duelo da Realeza & Invasão da Equipe Rocket: Eventos Duplos no Pokémon GO! 🔥"
> 
> 2.  **Nidos:** "Nidoqueen e Nidoking de Coroa" ficou fofo e liga ao tema do evento, mas o termo oficial para as versões raras é "Brilhantes" (Shiny, em inglês). Para mantermos a precisão, sugiro usar o termo mais comum e oficial.
>     *   Sugestão: "Nidoqueen e Nidoking Brilhantes: ✨ Já pensou em ter essas majestades brilhantes? 🤩 A hora é agora!" (O resto da frase continua ótimo!)
> 
> 3.  **Frustração:** A parte sobre o Ataque Frustração está ótima, super direta e com a energia certa ("NUNCA MAIS!" pegou bem!). Não precisa mudar.
> 
> 4.  **"Descubra como":** Mencionar que dá para aprender a evoluir o Bisharp e remover o Frustração sem explicar *como* é padrão para posts que direcionam para outro conteúdo (blog, vídeo, etc.). Se for o caso, está perfeito para gerar curiosidade!
> 
> No geral, o rascunho já está forte e engajador. Ajustando esses pequenos pontos de clareza e precisão, ele fica nota 10 e prontíssimo para bombar no feed! 💪
> 
> Me diga o que acha das sugestões!
